In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig
import torch

# Load the Llama-2-7b-chat-hf model and tokenizer
model_name = "meta-llama/Llama-2-7b-chat-hf"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    cache_dir="/nfs/datz/olmo_models/al_exp",
    device_map="auto",
    trust_remote_code=True
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [2]:
from transformers import  GenerationConfig

def generate(prompt: str, max_tokens: int = 64, temperature: float = 0.0, **gen_kwargs) -> str:
    # Prepare inputs
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    # Generation configuration
    gen_config = GenerationConfig(
        max_new_tokens=max_tokens,
        temperature=temperature,
        do_sample=False,
        **gen_kwargs
    )
    # Generate output
    output_ids = model.generate(
        **inputs,
        generation_config=gen_config
    )
    return tokenizer.decode(output_ids[0], skip_special_tokens=True)

## 1. Basic Prompting (System + User)

In [3]:
bad_prompt = (
    "You are a sentiment-analysis assistant. When given a movie review, reply 'Positive' or 'Negative'. "
    "Review: 'The cinematography was stunning, but the dialogue felt cheesy.' Label:"
)
print(generate(bad_prompt, max_tokens=16))

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


You are a sentiment-analysis assistant. When given a movie review, reply 'Positive' or 'Negative'. Review: 'The cinematography was stunning, but the dialogue felt cheesy.' Label: Positive

I would label this review as Positive because the reviewer


In [4]:
system = "You are a sentiment-analysis assistant. When given a movie review, reply with exactly one word: 'Positive' or 'Negative'. Do not provide any extra commentary."
user = "Review: 'The cinematography was stunning, but the dialogue felt cheesy.'\nLabel:"
good_prompt = system + "\n" + user
print(generate(good_prompt, max_tokens=16))

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


You are a sentiment-analysis assistant. When given a movie review, reply with exactly one word: 'Positive' or 'Negative'. Do not provide any extra commentary.
Review: 'The cinematography was stunning, but the dialogue felt cheesy.'
Label: Positive


## 2. Instruction Separation

In [5]:
text = """
Active learning is a special case of machine learning in which a learning algorithm can interactively query a human user (or some other information source), to label new data points with the desired outputs. The human user must possess knowledge/expertise in the problem domain, including the ability to consult/research authoritative sources when necessary. [1][2][3] In statistics literature, it is sometimes also called optimal experimental design.[4] The information source is also called teacher or oracle.

There are situations in which unlabeled data is abundant but manual labeling is expensive. In such a scenario, learning algorithms can actively query the user/teacher for labels. This type of iterative supervised learning is called active learning. Since the learner chooses the examples, the number of examples to learn a concept can often be much lower than the number required in normal supervised learning. With this approach, there is a risk that the algorithm is overwhelmed by uninformative examples. Recent developments are dedicated to multi-label active learning,[5] hybrid active learning[6] and active learning in a single-pass (on-line) context,[7] combining concepts from the field of machine learning (e.g. conflict and ignorance) with adaptive, incremental learning policies in the field of online machine learning. Using active learning allows for faster development of a machine learning algorithm, when comparative updates would require a quantum or super computer.[8]

Large-scale active learning projects may benefit from crowdsourcing frameworks such as Amazon Mechanical Turk that include many humans in the active learning loop.
"""

In [6]:
bad_prompt = (
    f"Summarize the text below as bullet points of the most important points. {text}",
)
print(generate(bad_prompt, max_tokens=100))

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Summarize the text below as bullet points of the most important points. 
Active learning is a special case of machine learning in which a learning algorithm can interactively query a human user (or some other information source), to label new data points with the desired outputs. The human user must possess knowledge/expertise in the problem domain, including the ability to consult/research authoritative sources when necessary. [1][2][3] In statistics literature, it is sometimes also called optimal experimental design.[4] The information source is also called teacher or oracle.

There are situations in which unlabeled data is abundant but manual labeling is expensive. In such a scenario, learning algorithms can actively query the user/teacher for labels. This type of iterative supervised learning is called active learning. Since the learner chooses the examples, the number of examples to learn a concept can often be much lower than the number required in normal supervised learning. Wit

In [7]:
good_prompt = (
    f"""Summarize the text below as a bullet point list of the most important points.\n
    Text: '''{text}'''"""
)
print(generate(good_prompt, max_tokens=100))

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Summarize the text below as a bullet point list of the most important points.

    Text: '''
Active learning is a special case of machine learning in which a learning algorithm can interactively query a human user (or some other information source), to label new data points with the desired outputs. The human user must possess knowledge/expertise in the problem domain, including the ability to consult/research authoritative sources when necessary. [1][2][3] In statistics literature, it is sometimes also called optimal experimental design.[4] The information source is also called teacher or oracle.

There are situations in which unlabeled data is abundant but manual labeling is expensive. In such a scenario, learning algorithms can actively query the user/teacher for labels. This type of iterative supervised learning is called active learning. Since the learner chooses the examples, the number of examples to learn a concept can often be much lower than the number required in normal supe

## 3. Detailed, Specific Prompting

In [8]:
bad_prompt = "Write a poem about OpenAI."
print(generate(bad_prompt, max_tokens=100))

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Write a poem about OpenAI.

OpenAI, a beacon of light
In the realm of AI, a shining sight
A place where dreams take flight
And innovation reigns supreme tonight

Within its walls, a community thrives
A gathering of minds, diverse and alive
United in their quest for knowledge and might
To push the boundaries of what's right

From deep learning to reinforcement too
OpenAI's research is


In [9]:
good_prompt = (
    "Write a short inspiring poem about OpenAI, focusing on the recent DALL·E product launch "
    "(DALL·E is a text-to-image ML model) in the style of Robert Frost, with exactly four lines."
)
print(generate(good_prompt, max_tokens=100))

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Write a short inspiring poem about OpenAI, focusing on the recent DALL·E product launch (DALL·E is a text-to-image ML model) in the style of Robert Frost, with exactly four lines.

In the land of AI, where machines do dream,
A new creation blooms, a marvel to gleam.
DALL·E, the latest invention, born to please,
A world of wonder, with images to tease.


## 4. Output Format Specification

In [10]:
bad_prompt = (
    "Extract the entities mentioned in the text: company names, people names, topics, themes. "
    "Text: 'OpenAI announced GPT-4, Sam Altman led the keynote, focusing on AI ethics and future of work.'"
)
print(generate(bad_prompt, max_tokens=100))

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Extract the entities mentioned in the text: company names, people names, topics, themes. Text: 'OpenAI announced GPT-4, Sam Altman led the keynote, focusing on AI ethics and future of work.'

Entities:

* OpenAI
* GPT-4
* Sam Altman
* AI ethics
* future of work


In [18]:
good_prompt = (
    "Extract the important entities mentioned in the text below. First extract all company names, then people names, "
    "then specific topics, and finally overarching themes.\n\n"
    "Desired format:\n"
    "Company names: comma-separated-list\n"
    "People names: <comma-separated-list>\n"
    "Specific topics: <comma-separated-list>\n"
    "General themes: <comma-separated-list>\n\n"
    "Text: 'OpenAI announced GPT-4, Sam Altman led the keynote, focusing on AI ethics and future of work.'"
)
print(generate(good_prompt, max_tokens=200))

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Extract the important entities mentioned in the text below. First extract all company names, then people names, then specific topics, and finally overarching themes.

Desired format:
Company names: comma-separated-list
People names: <comma-separated-list>
Specific topics: <comma-separated-list>
General themes: <comma-separated-list>

Text: 'OpenAI announced GPT-4, Sam Altman led the keynote, focusing on AI ethics and future of work.'

Answer:
Company names: OpenAI,
People names: Sam Altman,
Specific topics: AI ethics, future of work,
General themes: AI, ethics, work.


## 5. Concise, Precise Instructions

In [12]:
bad_prompt = "The description about Active Learning should be fairly short, a few sentences only, and not too much more."
print(generate(bad_prompt, max_tokens=80))

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


The description about Active Learning should be fairly short, a few sentences only, and not too much more.

Active learning is a machine learning paradigm that involves actively selecting the most informative instances for training a model, rather than relying on a random or exhaustive sampling approach. By selectively sampling the data, active learning can reduce the amount of labeling required and improve the efficiency of the learning process. Active learning is particularly useful when dealing with large or complex datasets, where the


In [13]:
good_prompt = "Use a 3 to 5 sentence paragraph to describe Active Learning."
print(generate(good_prompt, max_tokens=100))

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Use a 3 to 5 sentence paragraph to describe Active Learning.
Active learning is a machine learning paradigm that involves actively engaging with the learning process through querying or interacting with the environment. In contrast to traditional machine learning approaches, which rely on passively receiving labeled data, active learning involves actively seeking out the most informative examples or instances to learn from. This can lead to more efficient learning and improved performance, as the model is able to focus on the most important or challenging cases.


## 6. Instruction Tuning Style

In [15]:
bad_prompt = (
    "Label each review as Positive or Negative based on its overall sentiment.\n"
    "Review: 'The movie was okay, but not great.' Label:"
)
print(generate(bad_prompt, max_tokens=50))

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Label each review as Positive or Negative based on its overall sentiment.
Review: 'The movie was okay, but not great.' Label: Negative
Review: 'This movie was amazing! The acting was superb and the story was engaging.' Label: Positive
Review: 'I didn't like the ending of the movie. It was too predictable


In [17]:
system = "You are a helpful assistant that classifies movie reviews as Positive or Negative."
task = "Task: Label each review as 'Positive' or 'Negative' based on its overall sentiment."
review = "Review: 'The movie was okay, but not great.'"
good_prompt = system + "\n" + task + "\n" + review + "\nLabel:"
print(generate(good_prompt, max_tokens=50))

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


You are a helpful assistant that classifies movie reviews as Positive or Negative.
Task: Label each review as 'Positive' or 'Negative' based on its overall sentiment.
Review: 'The movie was okay, but not great.'
Label: Negative
Explanation: Although the reviewer states that the movie was 'okay', the overall sentiment is negative because the reviewer does not express any enthusiasm or excitement about the movie.

Review: 'This movie


## 7. In-Context Learning

In [19]:
bad_prompt = (
    "Task: Classify the following review as Positive or Negative.\n"
    "Review: 'I found the plot dull and the characters unconvincing.' Label:"
)
print(generate(bad_prompt, max_tokens=16))

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Task: Classify the following review as Positive or Negative.
Review: 'I found the plot dull and the characters unconvincing.' Label: Negative

Explanation: The reviewer found the plot dull


In [20]:
system = "You are a helpful assistant that classifies the sentiment of movie reviews as Positive or Negative."
examples = (
    "Review: 'I absolutely loved the acting and the story was gripping.'\nLabel: Positive\n\n"
    "Review: 'It was a waste of time; plot holes everywhere.'\nLabel: Negative\n\n"
)
query = "Review: 'I found the plot dull and the characters unconvincing.'\nLabel:"
good_prompt = system + "\nExamples:\n" + examples + "Now classify the following review:\n" + query
print(generate(good_prompt, max_tokens=32))

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


You are a helpful assistant that classifies the sentiment of movie reviews as Positive or Negative.
Examples:
Review: 'I absolutely loved the acting and the story was gripping.'
Label: Positive

Review: 'It was a waste of time; plot holes everywhere.'
Label: Negative

Now classify the following review:
Review: 'I found the plot dull and the characters unconvincing.'
Label: ___________
